## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import os

import sys
sys.path.append('..')

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 

data_file = os.path.join("..","Weather_Data","WeatherPy_database.csv")
city_data_df = pd.read_csv(data_file)
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Punta Arenas,CL,-53.1500,-70.9167,46.51,66,40,6.91,scattered clouds
1,1,Victoria,HK,22.2855,114.1577,86.18,84,100,3.00,overcast clouds
2,2,Dikson,RU,73.5069,80.5464,39.90,96,100,19.22,overcast clouds
3,3,Port Alfred,ZA,-33.5906,26.8910,54.18,90,37,5.01,scattered clouds
4,4,San Patricio,US,28.0170,-97.5169,91.29,64,6,14.25,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 73


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) &
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
44,44,Rikitea,PF,-23.1203,-134.9692,72.23,82,100,25.93,light rain
56,56,Thompson,CA,55.7435,-97.8558,71.76,88,40,3.44,scattered clouds
65,65,Hami,CN,42.8000,93.4500,70.38,23,2,11.16,clear sky
69,69,Horadiz,AZ,39.4488,47.3353,71.96,41,0,3.76,clear sky
72,72,Moroni,KM,-11.7022,43.2551,72.25,68,1,5.12,clear sky


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df = preferred_cities_df.dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
44,Rikitea,PF,72.23,light rain,-23.1203,-134.9692,
56,Thompson,CA,71.76,scattered clouds,55.7435,-97.8558,
65,Hami,CN,70.38,clear sky,42.8000,93.4500,
69,Horadiz,AZ,71.96,clear sky,39.4488,47.3353,
72,Moroni,KM,72.25,clear sky,-11.7022,43.2551,
78,Hargeysa,SO,70.74,few clouds,9.5600,44.0650,
102,Sao Joao Da Barra,BR,71.74,clear sky,-21.6403,-41.0511,
112,Arraial Do Cabo,BR,70.74,clear sky,-22.9661,-42.0278,
119,Ponta Do Sol,PT,71.01,scattered clouds,32.6667,-17.1000,
121,Gamba,GA,72.30,broken clouds,-2.6500,10.0000,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}     
        
# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try: hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except: print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df=hotel_df.dropna()

In [16]:
# 8a. Create the output File (CSV)
output_data_file = "../weather_data/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [15]:
# 11a. Add a marker layer for each city to the map. 
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))